In [ ]:
de v1 à v2 on introduit les masks wm et csf et les meants <br>
de v2 à v3 on plot les mocos, on wrappe le code python, on wrappe le code mathlab bramilla. on refait tout le pipeline <br>
de v3 à v4 on change les premier et dernier noeud en identity interfacepour se brancher direct sur le proprocess pipeline <br>
v5 au retour de hackfest on change les dérivées t - (t-1) <br>
on fait pour les regresseurs R R' R2 plus les 3 tissus dependants soit 21 regresseurs <br>
v6 on prend les résultats obtenus avec le preprocess corrigé <br>
v7 le preprocess a été changé imagecalculator est passé de nibabel à fslmaths <br>


# To calculate parameters for afni bandpass function and write them in ortho file.txt


# Start of the new subworkflow - PREBANDPASS

In [1]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface

from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces.fsl.utils import PlotMotionParams   # to plot moco variables

In [2]:
# creation of a subworflow to calculate the bandpass parameters
prebandpass = pe.Workflow(name='prebandpass')

In [3]:
# todo remplace this node by an identity node that get input from preprocess pipeline / node 7 eg. fsl merge
# input node get the good files first the tissues normalised files
# the files to merge arein /scratch/user/hirsch/datadir/data_results/functionnal
sourcedir = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files'
# second the merged functionnal file
sourcemergeddir = '//scratch/user/hirsch/datadir4/data_results_py/functionnal/normalized_files'
funcdir = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/realignment_parameters'


from nipype import SelectFiles, Node
templates = dict(wmMask=sourcedir+ "/" + "wc2*.nii",
                 csfMask=sourcedir+ "/" + "wc3*.nii",
                 filesToMerge=sourcemergeddir + "/" + "wcra*.nii",
                 mocoVariables=funcdir+ "/" + "rp*.txt")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()
hpass =  0.005
lpass = 0.2


## 1 - compute moco file to feed with ortho.txt file the bandpass node of preprocess workflow 

## Node 1 - compute moco

In [5]:

def computeMoco(mocoFile):
    import pandas as pd
    import numpy as np
    import os
    

    # read the moco file to put it in a panda dataframe 
    mocodf = pd.read_csv(mocoFile, header=None, sep='  ',engine='python')
    print(mocodf.head())
    # todo recuperer ces infos de l'autre pipeline
    TR = 2000 
    acqNb = 240

    def vectorDerivative(v):
        dv = {}
        for i in range(acqNb):
            # print mocodf['x'][i]
            if i== 0:
                dv[i]= 0
            elif i== acqNb-1:
                dv[i]= v[i]-v[i-1]
            else:
                dv[i]= v[i]-v[i-1]
                #print 'derivative' + str(i)
                #print  v[i]
        return v

    def plusDerivative(df):
        lg = len(df.columns.values)
        dg = df
        for j in list(df.columns.values):
            vprime = vectorDerivative(df[j])
            dg[lg+j]=vprime
        return dg
    
    def plusSquare(df):
        lg = len(df.columns.values)
        ds = df
        for j in range(6):
            print j
            vs = df[j]**2
            ds[lg+j]=vs
        return ds    
    

    
    # first, we derivate the 6 colunms of dataframe of moco file, and append the 6 new colums to df
    dfderivate = plusDerivative(mocodf)
    print(dfderivate.head())
    
    # then, we compute the square of the 6 first colums, and append them to df. it makes 18 colums that are going to 
    # participate in the ortho file to make 18 regressors bandpassed
    dfsquare = plusSquare(dfderivate)
    
    g = dfsquare.to_csv('ortho.txt', sep=' ', index=False,header=False)
    print g
    h = os.getcwd() + '/' + 'ortho.txt'
    return h
    

computeMoco = Node(Function(input_names=['mocoFile'],
                                output_names=['out_file'],
                                function=computeMoco),
                                name='computeMoco')



prebandpass.connect(filesource, "mocoVariables", computeMoco, "mocoFile")

##  2 - Moco plots

In [6]:
# plot moco variables rotations
MotionCorrectionPlot1 = Node(PlotMotionParams(), name="MotionCorrectionPlot1")
MotionCorrectionPlot1.inputs.ignore_exception = False     
MotionCorrectionPlot1.inputs.in_source = 'spm'     
MotionCorrectionPlot1.inputs.output_type = 'NIFTI_GZ'     
MotionCorrectionPlot1.inputs.plot_size = (500, 1000)     
MotionCorrectionPlot1.inputs.plot_type = 'rotations'     
MotionCorrectionPlot1.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource, "mocoVariables", MotionCorrectionPlot1, "in_file")

In [7]:
# plot moco variables translations
MotionCorrectionPlot2 = Node(PlotMotionParams(), name="MotionCorrectionPlot2")
MotionCorrectionPlot2.inputs.ignore_exception = False     
MotionCorrectionPlot2.inputs.in_source = 'spm'     
MotionCorrectionPlot2.inputs.output_type = 'NIFTI_GZ'     
MotionCorrectionPlot2.inputs.plot_size = (500, 1000)     
MotionCorrectionPlot2.inputs.plot_type = 'translations'     
MotionCorrectionPlot2.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource, "mocoVariables", MotionCorrectionPlot2, "in_file")

In [8]:
# plot moco variables displacement
MotionCorrectionPlot3 = Node(PlotMotionParams(), name="MotionCorrectionPlot3")
MotionCorrectionPlot3.inputs.ignore_exception = False     
MotionCorrectionPlot3.inputs.in_source = 'spm'     
MotionCorrectionPlot3.inputs.output_type = 'NIFTI_GZ'     
MotionCorrectionPlot3.inputs.plot_size = (500, 1000)     
MotionCorrectionPlot3.inputs.plot_type = 'displacement'     
MotionCorrectionPlot3.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource, "mocoVariables", MotionCorrectionPlot3, "in_file")

## 3 - get wm and csf mask (normalised), erode them and calculate signal mean on both masks
(input from segment + normalyse wmMask = '/homes_unix/hirsch/_pypipe/datadir/data_results/structural/norm_files/wc2t0009_t1_s03.nii')
remark: erosion is done once; in connectomics, it is done 3 times

In [9]:
# calculate eroded binary mask for wm
from nipype.interfaces.fsl.maths import MathsCommand

erosion = pe.Node(interface=MathsCommand(), name='erosion')
    
erosion.inputs.args = '-thr 0 -uthr 111 -bin -ero  '     
erosion.inputs.ignore_exception = False     
erosion.inputs.output_type = 'NIFTI_GZ'     
erosion.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"wmMask" , erosion, "in_file")

In [10]:
# calculate eroded binary mask for lcf

erosioncsf = pe.Node(interface=MathsCommand(), name='erosioncsf')
    
erosioncsf.inputs.args = '-thr 0 -uthr 111 -bin -ero '     
erosioncsf.inputs.ignore_exception = False     
erosioncsf.inputs.output_type = 'NIFTI_GZ'     
erosioncsf.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"csfMask" , erosioncsf, "in_file")

In [11]:
# lets merge the functionnal normalysed files with fsl merge

from nipype.interfaces.fsl import Merge
fsl_merge = pe.Node(interface=Merge(), name="fsl_merge")     
 
fsl_merge.inputs.dimension = 't'
fsl_merge.inputs.output_type = 'NIFTI_GZ'
# todo recuperer le TR deja exploité du xml
fsl_merge.inputs.tr = 2.0
#fsl_merge.inputs.force_even = False     
fsl_merge.inputs.ignore_exception = False     
#fsl_merge.inputs.sort = False

prebandpass.connect(filesource,  'filesToMerge', fsl_merge, 'in_files')

In [12]:
# lets calculate the mean signal on these eroded masks first on wm
from nipype.interfaces.fsl.utils import ImageMeants

wmMeants = Node(ImageMeants(), name="wmMeants")     
wmMeants.inputs.ignore_exception = False     
wmMeants.inputs.order = 1     
wmMeants.inputs.output_type = 'NIFTI_GZ'     
wmMeants.inputs.terminal_output = 'stream'     
prebandpass.connect(fsl_merge, "merged_file" , wmMeants, "in_file")   
prebandpass.connect(erosion, "out_file", wmMeants, "mask")


In [13]:
# lets calculate the mean signal on these eroded masks econd on csf

csfMeants = Node(ImageMeants(), name="csfMeants")     
csfMeants.inputs.ignore_exception = False     
csfMeants.inputs.order = 1     
csfMeants.inputs.output_type = 'NIFTI_GZ'     
wmMeants.inputs.terminal_output = 'stream'     
prebandpass.connect(fsl_merge, "merged_file" , csfMeants, "in_file")   
prebandpass.connect(erosioncsf, "out_file", csfMeants, "mask")


In [14]:
from ginnipi.toolbox.computations import  mergeTables
from ginnipi.toolbox.flow import createList3

# Node: func2.createListFrom3
createListFrom3 = Node(Function(input_names=['item1', 'item2', 'item3'],                                     
                                output_names=['out_list'],                                     
                                function=createList3),                            
                                name="createListFrom3")     
createListFrom3.inputs.ignore_exception = False     
prebandpass.connect(computeMoco, "out_file", createListFrom3, "item1")     
prebandpass.connect(wmMeants, "out_file", createListFrom3, "item2")     
prebandpass.connect(csfMeants, "out_file", createListFrom3, "item3")         

# Node: func2.buildNuisanceTable     
buildNuisanceTable = Node(Function(input_names= ['in_files'],                                        
                                   output_names=['out_file'],                                        
                                   function=mergeTables),                               
                                   name="buildNuisanceTable")     
buildNuisanceTable.inputs.ignore_exception = False     
prebandpass.connect(createListFrom3, "out_list", buildNuisanceTable, "in_files")

In [15]:
# lets make a list from the 3 file moco+derivative+square, wmmeants, csfmeants



In [16]:
# lets calculate the band pass with 3 ortho files moco+derivative+square, wm_meants and csf_meants
from nipype.interfaces import afni

afniBandpass = pe.Node(interface=afni.Bandpass(), name='afniBandpass')
  
afniBandpass.inputs.automask = False     
afniBandpass.inputs.environ = {}     
afniBandpass.inputs.highpass = hpass     
afniBandpass.inputs.ignore_exception = False     
afniBandpass.inputs.lowpass = lpass     
afniBandpass.inputs.outputtype = 'NIFTI_GZ'     
afniBandpass.inputs.terminal_output = 'stream'   

prebandpass.connect(fsl_merge, "merged_file", afniBandpass, "in_file")
prebandpass.connect(buildNuisanceTable, "out_file", afniBandpass, "orthogonalize_file")        



In [17]:
# lets make an output node with the 3 files to be pass to afni bandpass: 
# file 1 with 1 colums moco, derivative moco, moco and derivative square
# file 2 with wm mean signal on wm
# file 3 with lcf mean signal on lcf

field_list=['rotations_plot', 
            'translations_plot',
            'displacement_plot',
            'moco_gradient_square',
            'wm_normalized_eroded_mask',
            'wm_meants',
            'csf_normalized_eroded_mask',
            'csf_meants',
            "merged_file",
            'bandpassedFile']
            
            
outputNode = Node(IdentityInterface(fields=field_list), name="outputNode")

# for plot files
prebandpass.connect(MotionCorrectionPlot1,  'out_file', outputNode, 'rotations_plot')
prebandpass.connect(MotionCorrectionPlot2,  'out_file', outputNode, 'translations_plot')
prebandpass.connect(MotionCorrectionPlot3,  'out_file', outputNode, 'displacement_plot')

# for moco , gradient and square
prebandpass.connect(computeMoco,  'out_file', outputNode, 'moco_gradient_square')

# for segmented normalised eroded wm and lcf mask
prebandpass.connect(erosion,  'out_file', outputNode, 'wm_normalized_eroded_mask')
prebandpass.connect(erosioncsf,  'out_file', outputNode, 'csf_normalized_eroded_mask')

# for wm and csf mean signal to text files in functionnal repository
prebandpass.connect(wmMeants,  'out_file', outputNode, 'wm_meants')
prebandpass.connect(csfMeants,  'out_file', outputNode, 'csf_meants')

prebandpass.connect(fsl_merge, "merged_file", outputNode, "merged_file")

prebandpass.connect(afniBandpass,  'out_file', outputNode, 'bandpassedFile')

In [18]:
# the run
prebandpass.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpWyFD78/prebandpass/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node erosion in dir: /tmp/tmpbuQQNR/prebandpass/erosion
INFO:workflow:Running: fslmaths /scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc2t0009_t1_s03.nii -thr 0 -uthr 111 -bin -ero   /tmp/tmpbuQQNR/prebandpass/erosion/wc2t0009_t1_s03_maths.nii.gz
INFO:workflow:Executing node computeMoco in dir: /tmp/tmpN3UOIm/prebandpass/computeMoco
INFO:workflow:Executing node MotionCorrectionPlot1 in dir: /tmp/tmpkoTsi2/prebandpass/MotionCorrectionPlot1
INFO:workflow:Running: fsl_tsplot -i /scratch/user/hirsch/datadir4/data_results_py/functionnal/realignment_parameters/rp_at0009_epi_s04_d0001.txt -o /scratch/user/hirsch/datadir4/data_results_py/functionnal/realignment_parameters/rp_at0009_epi_s04_d0001_rot.png -h 500 -w 1000 -t 

              0         1         2             3             4             5
0  1.332086e-14  0.000000  0.000000 -6.237710e-52  2.402678e-85 -3.851860e-34
1  2.698024e-03  0.013874 -0.000347 -1.778364e-04 -2.922913e-05  8.624682e-05
2  5.508635e-03  0.009560 -0.009807 -4.335903e-05 -3.450885e-05  9.118062e-05
3  1.131352e-03  0.037336 -0.011560  1.258265e-04  6.879228e-05 -4.295511e-05
4  1.740320e-03  0.055928 -0.018120  9.074919e-05  1.142829e-04 -1.289218e-04
             0         1         2             3             4             5   \
0  1.332086e-14  0.000000  0.000000 -6.237710e-52  2.402678e-85 -3.851860e-34   
1  2.698024e-03  0.013874 -0.000347 -1.778364e-04 -2.922913e-05  8.624682e-05   
2  5.508635e-03  0.009560 -0.009807 -4.335903e-05 -3.450885e-05  9.118062e-05   
3  1.131352e-03  0.037336 -0.011560  1.258265e-04  6.879228e-05 -4.295511e-05   
4  1.740320e-03  0.055928 -0.018120  9.074919e-05  1.142829e-04 -1.289218e-04   

             6         7         8           

INFO:workflow:Executing node buildNuisanceTable in dir: /tmp/tmpyHUByn/prebandpass/buildNuisanceTable
INFO:workflow:Executing node afniBandpass in dir: /tmp/tmpWE71Du/prebandpass/afniBandpass
INFO:workflow:Running: 3dBandpass -prefix wcrat0009_epi_s04_d0001_merged_bp.nii.gz -ort /tmp/tmpyHUByn/prebandpass/buildNuisanceTable/merged_array.txt 0.005000 0.200000 /tmp/tmpWE71Du/prebandpass/afniBandpass/wcrat0009_epi_s04_d0001_merged.nii.gz
INFO:interface:stderr 2017-04-20T18:00:01.100450:++ 3dBandpass: AFNI version=AFNI_16.0.00 (Jan  1 2016) [64-bit]
INFO:interface:stderr 2017-04-20T18:00:01.100450:++ Authored by: RW Cox
INFO:interface:stderr 2017-04-20T18:00:01.100450:++ Data length = 240  FFT length = 240
INFO:interface:stderr 2017-04-20T18:00:01.100450: + bandpass: ntime=240 nFFT=240 dt=2 dFreq=0.00208333 Nyquist=0.25 passband indexes=2..96
INFO:interface:stderr 2017-04-20T18:00:01.100450:++ Loading input dataset time series
INFO:interface:stderr 2017-04-20T18:00:07.902136:++ No mask ==>

# End of precalculations dump in ortho.txt file

# Brouillons

In [19]:
%pwd

u'/homes_unix/hirsch/_ipnotebooks'

In [4]:
PlotMotionParams().help()

Wraps command **fsl_tsplot**

Use fsl_tsplot to plot the estimated motion parameters from a realignment
program.


Examples
--------

>>> import nipype.interfaces.fsl as fsl
>>> plotter = fsl.PlotMotionParams()
>>> plotter.inputs.in_file = 'functional.par'
>>> plotter.inputs.in_source = 'fsl'
>>> plotter.inputs.plot_type = 'rotations'
>>> res = plotter.run() #doctest: +SKIP


Notes
-----

The 'in_source' attribute determines the order of columns that are expected
in the source file.  FSL prints motion parameters in the order rotations,
translations, while SPM prints them in the opposite order.  This interface
should be able to plot timecourses of motion parameters generated from other
sources as long as they fall under one of these two patterns.  For more
flexibilty, see the :class:`fsl.PlotTimeSeries` interface.

Inputs::

	[Mandatory]
	in_file: (an existing file name or a list of items which are an
		 existing file name)
		file with motion parameters
		flag: %s, position: 1
	in_sourc

In [15]:
%env PYTHONPATH
%pwd



u'/homes_unix/hirsch/_ipnotebooks'

In [ ]:
%env PYTHONPATH = ${PYTHONPATH}:/homes_unix/hirsch/ginfizz/src:/homes_unix/hirsch/ginnipi

In [ ]:
from ginnipi.toolbox.computations import  mergeTables

In [ ]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'

# for plot files
prebandpass.connect(MotionCorrectionPlot1,  'out_file', datasink, '.plotfiles')

# for segmented normalised eroded wm and lcf mask
prebandpass.connect(erosion,  'out_file', datasink, 'functionnal.bandpass_wm_mask')
prebandpass.connect(erosionLcf,  'out_file', datasink, 'functionnal.bandpass_lcf_mask')

# for wm and lcf mean signal to text files in functionnal repository
prebandpass.connect(wmMeants,  'out_file', datasink, 'functionnal.bandpass_wm_meants')
prebandpass.connect(lcfMeants,  'out_file', datasink, 'functionnal.bandpass_lcf_meants')

In [ ]:
def computeOrtho(file1, file2, file3):
    fileList = []
    fileList.append(file1)
    fileList.append(file2)
    fileList.append(file3)
    return fileList

computeOrtho = Node(Function(input_names=['file1', 'file2', 'file3'],
                                output_names=['out_file'],
                                function=computeOrtho),
                                name='computeOrtho')

prebandpass.connect(computeMoco, "out_file", computeOrtho, 'file1')
prebandpass.connect(wmMeants, "out_file", computeOrtho, 'file2')
prebandpass.connect(csfMeants, "out_file", computeOrtho, 'file3')

In [ ]:
%pwd


In [ ]:
mocoFile = '/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal/rp_at0009_epi_s04_d0001.txt'


In [ ]:
mocoOrtho('/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal/rp_at0009_epi_s04_d0001.txt')

In [ ]:
ImageMeants().help()

In [ ]:
%cd '/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal'
%pwd

In [ ]:
cwd = os.getcwd()
print cwd

In [ ]:
square = plusSquare(derivate)
print(square.head())

In [ ]:
vectorSquare(mocodf[0])

In [ ]:
list(mocodf.columns.values)

In [ ]:
vectorderivative(mocodf[0])

In [ ]:
sdx = pd.Series(dx)

In [ ]:
sdx